In [35]:
import django, os, sys, time, resource, re, gc, shutil
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from django.db.models import Count, Avg
from multiprocess import Pool
from functools import partial
import numpy as np
from functools import partial
from scipy.sparse import coo_matrix, csr_matrix, find, tril
import networkx as nx
import csv

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from tmv_app.models import *
qid = 1457

In [2]:
runs = [282, 283, 284, 285, 287]


fname = "../../data/topic_data/run_{}.xlsx".format(runs[0])

writer = pd.ExcelWriter(fname)

for i in range(1,len(runs)):

    topics1 = DynamicTopic.objects.filter(run_id=runs[i-1])
    topics2 = DynamicTopic.objects.filter(run_id=runs[i])

    df = pd.DataFrame.from_dict(list(topics2.values('title','id')))
    df['similarity'] = 0
    df['stopic'] = ""

    for t in topics2:
        scores = [0]
        titles = [""]
        for ct in topics1:
            score = len(set(t.top_words).intersection(set(ct.top_words)))
            if score>0:
                scores.append(score)
                titles.append(ct.title)

        
        m = max(scores)
        df.loc[df['id']==t.id,'similarity'] = m
        df.loc[df['id']==t.id,'stopic'] = titles[scores.index(max(scores))]


    df = df.sort_values('similarity',ascending=False)
    df.head()
    
    
    

    
    df.to_excel(writer,"run_{}".format(runs[i]))
    
writer.save()

In [3]:
runs = list(range(295,300))

writer = pd.ExcelWriter(fname)
fname = "../../data/topic_data/run_{}.xlsx".format(runs[0])

for i in range(1,len(runs)):

    topics1 = Topic.objects.filter(run_id=runs[i-1])
    topics2 = Topic.objects.filter(run_id=runs[i])

    df = pd.DataFrame.from_dict(list(topics2.values('title','id')))
    df['similarity'] = 0
    df['stopic'] = ""

    for t in topics2:
        scores = [0]
        titles = [""]
        for ct in topics1:
            score = len(set(t.top_words).intersection(set(ct.top_words)))
            if score>0:
                scores.append(score)
                titles.append(ct.title)

        
        m = max(scores)
        df.loc[df['id']==t.id,'similarity'] = m
        df.loc[df['id']==t.id,'stopic'] = titles[scores.index(max(scores))]


    df = df.sort_values('similarity',ascending=False)
    df.head()
    
    
    
    
    

    
    df.to_excel(writer,"run_{}".format(runs[i]))
    
writer.save()

In [61]:
runs = list(range(323,330))


#runs = [295,296,297]

col1s = []
col2s = []
ss = []
scols = []

for i in range(1,len(runs)):

    topics1 = Topic.objects.filter(run_id=runs[i-1])
    topics2 = Topic.objects.filter(run_id=runs[i])
    

    df = pd.DataFrame.from_dict(list(topics2.values('title','score')))
    
    df2 = pd.DataFrame.from_dict([{'title': 'None','score': 0}])
    
    df = df.append(df2)

    
    
    col1 = "run_{}_topics_{}".format(runs[i-1],topics1.count())
    
    scol = "scores_{}".format(runs[i])
    bscol = "scores_{}".format(runs[i-1])
    
    if i==1:
        scols.append(bscol)
    
    col1s.append(col1)
    scols.append(scol)
    
    col2 = "run_{}_topics_{}".format(runs[i], topics2.count())
    
    col2s.append(col2) 
    
    s = "similarity_{}-{}".format(runs[i-1],runs[i])
    ss.append(s)
    
    df = df.rename(columns = {'title': col2, 'score': scol})

    df[s] = 0
    df[col1] = "None"
    df[bscol] = 0

    for t in topics2:
        scores = [0]
        titles = [""]
        tscores = [0]
        for ct in topics1:
            score = len(set(t.top_words).intersection(set(ct.top_words)))
            if score>0:
                scores.append(score)
                titles.append(ct.title)
                tscores.append(ct.score)

        
        m = max(scores)
        df.loc[df[col2]==t.title, s] = m
        if m==0:
            df.loc[df[col2]==t.title, col1] = 'None'
        else:
            df.loc[df[col2]==t.title, col1] = titles[scores.index(max(scores))]
            df.loc[df[col2]==t.title, bscol] = tscores[scores.index(max(scores))]
            

    if i==1:   
        #df = pd.DataFrame.from_dict(list(topics2.values('title')))
        mdf = df
    else:
        mdf = mdf.merge(df,how="outer").fillna("")
    

columns = []
for i in range(len(col1s)):
    columns.append(col1s[i])
    columns.append(scols[i])
    columns.append(ss[i])
    if i == len(col1s)-1:
        columns.append(col2s[i])
        columns.append(scols[i+1])

print(columns)

mdf = mdf[columns]

mdf.head()



['run_323_topics_50', 'scores_323', 'similarity_323-324', 'run_324_topics_60', 'scores_324', 'similarity_324-325', 'run_325_topics_70', 'scores_325', 'similarity_325-326', 'run_326_topics_80', 'scores_326', 'similarity_326-327', 'run_327_topics_90', 'scores_327', 'similarity_327-328', 'run_328_topics_100', 'scores_328', 'similarity_328-329', 'run_329_topics_110', 'scores_329']


,run_323_topics_50,scores_323,similarity_323-324,run_324_topics_60,scores_324,similarity_324-325,run_325_topics_70,scores_325,similarity_325-326,run_326_topics_80,scores_326,similarity_326-327,run_327_topics_90,scores_327,similarity_327-328,run_328_topics_100,scores_328,similarity_328-329,run_329_topics_110,scores_329
0,"{energi, renew, sourc}",65.348219,9,"{energi, renew, sourc}",60.620645,9,"{energi, renew, sourc}",57.0342,9,"{energi, renew, sourc}",55.1589,8,"{energi, renew, sourc}",52.2565,10,"{energi, renew, sourc}",49.7845,10,"{energi, renew, sourc}",48.4088
1,"{soil, eros, nutrient}",47.598966,10,"{soil, eros, tillag}",45.371670,9,"{soil, organ, eros}",42.2718,9,"{soil, eros, tillag}",38.6271,10,"{soil, eros, tillag}",36.2798,9,"{soil, eros, tillag}",33.9568,9,"{soil, eros, tillag}",32.1249
2,"{soil, eros, nutrient}",47.598966,10,"{soil, eros, tillag}",45.371670,9,"{soil, organ, eros}",42.2718,2,"{organ, convent, organiz}",19.8314,10,"{organ, convent, organiz}",16.7004,10,"{organ, convent, organiz}",23.8379,10,"{organ, convent, organiz}",16.0074
3,"{forest, manag, tree}",24.433050,9,"{forest, tree, forestri}",19.603212,10,"{forest, tree, forestri}",17.0827,8,"{forest, forestri, timber}",14.257,9,"{forest, forestri, timber}",13.563,9,"{forest, forestri, timber}",12.0663,9,"{forest, forestri, timber}",11.0827
4,"{forest, manag, tree}",24.433050,9,"{forest, tree, forestri}",19.603212,10,"{forest, tree, forestri}",17.0827,3,"{tree, stand, regener}",29.8846,9,"{tree, stand, regener}",27.9016,9,"{tree, stand, regener}",18.5534,10,"{tree, stand, regener}",14.0678


In [62]:
#col1s.append(col2)


#mdf = mdf.sort_values(columns,ascending=False).reset_index()


res = mdf.groupby(columns)



res = res.apply(lambda x: x.sort_values(s,ascending=False))

l = len(res)




fname = "../../data/topic_data/run_compare_{}_{}.xlsx".format(runs[0],runs[len(runs)-1])

writer = pd.ExcelWriter(fname, engine='xlsxwriter')

res.to_excel(writer)

worksheet = writer.sheets['Sheet1']

for i in range(len(ss)):
    if (i+1)*3 > 26:
        c = 'A'+chr(ord('A')-1+((i+1)*3)-26)
    else:
        c = chr(ord('A')-1+(i+1)*3)
    r = "{}2:{}{}".format(c,c,len(res))
    print(r)
    
    worksheet.conditional_format(r, {
        'type': '3_color_scale',
        'min_value': 0,
        'mid_value': 5,
        'max_value': 10,
        'min_type': 'num',
        'mid_type': 'num',
        'max_type': 'num',
    })

writer.save()

res.head() 
    
    
res.head()

C2:C131
F2:F131
I2:I131
L2:L131
O2:O131
R2:R131


run_323_topics_50  \
run_323_topics_50             scores_323 similarity_323-324 run_324_topics_60             scores_324 similarity_324-325 run_325_topics_70             scores_325 similarity_325-326 run_326_topics_80             scores_326       similarity_326-327 run_327_topics_90             scores_327       similarity_327-328 run_328_topics_100       scores_328       similarity_328-329 run_329_topics_110       scores_329                                            
None                          0.000000   0                  None                          0.000000   0.0                None                          0.000000   0.0                None                          0.0              0.0                None                          0.0              0.0                None                     0.0              0.0                None                     0.0              130                           None   
                                                                                                                                                                                                                                                                                                                                                                                     }                        0.0              129                           None   
{biodiesel, fuel, oil}        32.856837  8                  {biodiesel, oil, diesel}      22.034321  9.0                {biodiesel, oil, diesel}      20.055753  9.0                {biodiesel, oil, diesel}      15.1601915272664 9.0                {biodiesel, oil, diesel}      11.3260863648776 9.0                {biodiesel, oil, diesel} 10.7321774136785 9.0                {biodiesel, oil, diesel} 9.16976648185281 57          {biodiesel, fuel, oil}   
{biofuel, biomass, feedstock} 42.546358  9                  {biofuel, biomass, feedstock} 18.985342  6.0                {biofuel, feedstock, ethanol} 19.790973                                                                                                                                                                                                                                                                11   {biofuel, biomass, feedstock}   
                                                                                                     7.0                {biomass, bioenergi, potenti} 20.031538  7.0                {biofuel, biomass, feedstock} 20.7734511580175 9.0                {biofuel, biomass, feedstock} 5.56857845048985 2.0                {residu, wood, potenti}  17.1448696680322                                                              103  {biofuel, biomass, feedstock}   

                                                                                                                                                                                                                                                                                                                                                                                                                                    scores_323  \
run_323_topics_50             scores_323 similarity_323-324 run_324_topics_60             scores_324 similarity_324-325 run_325_topics_70             scores_325 similarity_325-326 run_326_topics_80             scores_326       similarity_326-327 run_327_topics_90             scores_327       similarity_327-328 run_328_topics_100       scores_328       similarity_328-329 run_329_topics_110       scores_329                         
None                          0.000000   0                  None                          0.000000   0.0                None                          0.000000   0.0                None                          0.0              0.0                None                          0.0              0.0                None                     0.0              0.0                None               

In [ ]:
print(mdf)

In [ ]:
runs = list(range(295,300))

runs = []

for i in range(len(runs)):

    topics = Topic.objects.filter(run_id=runs[i])
    
    tree = []
    
    for t in topics:
        
        tdict = {
            "parent_similarity": 0,
            "run": runs[i],
            "ntopics": topics.count(),
            "title": t.title,
            "sim_topics": [],
            "child_topics": 0
        }
        
        if i==0:
            tree.append(tdict)
        else:
            
            scores = [0]
            titles = [""]
            for ct in comparison_topics:
                score = len(set(t.top_words).intersection(set(ct.top_words)))
                if score>0:
                    scores.append(score)
                    titles.append(ct.title)


                parent = titles[scores.index(max(scores))]
                
                
                
                print(m)

In [ ]:
def find_similar(x):
    return(1,2)

df['bla'], df['x'] = zip(df.map(find_similar))
                         
df.head()